## Notebook Link
https://colab.research.google.com/drive/1uNygzDR4hISwLOgmDS31hRHfr6KAF7Ib?usp=sharing

In [ ]:
from IPython.display import clear_output
clear_output(wait=False)

!pip install Flask==2.3.2 Pillow==9.5.0 requests==2.30.0 flask-ngrok Flask-CORS==3.0.10 pyngrok transformers torch torchvision

clear_output(wait=False)
print("Setup complete!")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from PIL import Image
import io
import base64
from pyngrok import ngrok

# Configure ngrok
ngrok.set_auth_token('2oHUyMyGNJuD34GO6NdGJd8KAxd_3TyYzUGLMA9DvgUopNRw3')

app = Flask(__name__)
CORS(app)

# Global variables for models
image_captioner = None
story_tokenizer = None
story_model = None
device = None

def load_models():
    global image_captioner, story_tokenizer, story_model, device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    image_captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base", device=device)
    model_name = "gpt2"
    story_tokenizer = AutoTokenizer.from_pretrained(model_name)
    story_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.json
        image_data = data['image'].split(',')[1]
        genre = data['genre']

        # Process image
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))

        # Generate caption
        caption = image_captioner(image)[0]['generated_text']

        # Generate story
        prompt = f"Write a {genre.lower()} story about: {caption}\n\nStory:"
        inputs = story_tokenizer(prompt, return_tensors="pt").to(device)
        output_sequences = story_model.generate(
            input_ids=inputs['input_ids'],
            max_length=200,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1
        )

        story = story_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
        return jsonify({
            'success': True,
            'story': story.replace(prompt, '').strip()
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

if __name__ == '__main__':
    print("Loading AI models...")
    load_models()
    print("Models loaded successfully!")
    
    public_url = ngrok.connect(5000)
    api_url = f"{public_url.public_url}/generate_story"
    print(f'Server running at: {api_url}')
    
    app.run(port=5000)